In [3]:
import glob, sys

from omegaconf import OmegaConf
from tqdm.auto import tqdm
from grams.prelude import GRAMS, ROOT_DIR, DATA_DIR, I, O, M, WikidataSemanticModelHelper
sys.path.append(str(ROOT_DIR / "sm_annotator"))

In [4]:
cwd = ROOT_DIR / "examples/novartis"
cfg = OmegaConf.load(ROOT_DIR / "grams.yaml")
grams = GRAMS(DATA_DIR, cfg)

In [5]:
tables = [I.W2WTable.from_csv_file(infile) for infile in glob.glob(str(cwd / "tables/*.csv"))]

In [6]:
annotations = [grams.annotate(table) for table in tqdm(tables)]

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
from sm_annotator.prelude import Annotator, BatchAnnotator, widgets, GRAMSAnnotatorAssistant, SliderApp

qnodes = grams.qnodes.cache_dict()
wdprops = grams.wdprops
wdclasses = grams.wdclasses.cache_dict()

assistant = GRAMSAnnotatorAssistant([
    dict(table=table, sg=annotation.sg, dg=annotation.dg)
    for annotation, table in zip(annotations, tables)
], qnodes, wdprops)
annotator = Annotator(qnodes, wdclasses, wdprops, cwd / "ground-truth", 
                          eshost='http://mira.isi.edu:9200', username='', password='', dev=False, assistant=assistant)
batch_annotator = BatchAnnotator(annotator)
batch_annotator.render(same_tab=True)

Output()

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='a41c030b-87ec-4907-a86a-0d5d1d21ebce')

HTML(value='<div id="fa3da3bf-f122-4bd3-aeb3-57e7e6325633"></div>')

<IPython.core.display.Javascript object>

SlowTunnelWidget(js_endpoint=(0, ''), py_endpoint=(0, ''), tunnel_id='75c72064-7b37-4590-ba03-eebecd958e78')

<IPython.core.display.Javascript object>

In [9]:
batch_annotator.batch_annotate([
    (table.get_friendly_fs_id(), "", table)
    for table in tables
], start_index=0)